In [402]:
from __future__ import print_function
from __future__ import absolute_import

import datetime 
import numpy as np 
import pandas as pd

import keras
from keras import backend as K
from keras import Model 
from keras import regularizers
from keras.layers import Input, Dense, Lambda, LSTM
from keras.optimizers import Adam
from keras.losses import binary_crossentropy, mse
from keras.callbacks import TensorBoard, EarlyStopping

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [248]:
iowa_matches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_matches.csv')
iowa_nonmatches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_nonmatches.csv')

In [249]:
iowa_matches.head()

,uid1915,fname1915,lname1915,fullname1915,yob1915,hhid,fname1940,lname1940,fullname1940,yob1940,uid-hhid
0,uid0910071227,donald d,cutler,donald d cutler,1911,19067,donald dean,cutler,donald dean cutler,1911,uid0910071227-19067
1,uid0063131339,homer,taylor,homer taylor,1912,71505,homer ellis,taylor,homer ellis taylor,1912,uid0063131339-71505
2,uid0044088276,earl,stearnes,earl stearnes,1899,109708,earl,stearns,earl stearns,1900,uid0044088276-109708
3,uid0067053130,theodore,hornaday,theodore hornaday,1904,108304,theodore i,harnaday,theodore i harnaday,1904,uid0067053130-108304
4,uid0066046148,jack r,turner,jack r turner,1907,105092,jack r,turner,jack r turner,1907,uid0066046148-105092


In [619]:
import itertools 

def k_shingles(k):
    """
    Returns list of k-shingles out of 26 letters + space
    """
    letters = 'abcdefghijklmnopqrstuvwxyz '
    shingles = list(map(''.join, itertools.permutations(letters, k)))
    for letter in letters:
        shingles.append(letter * k)
    shingles.insert(0, '.  ')
    return shingles

pairs = k_shingles(2)

def embed_shingles(name, max_length, pairs=pairs,
                   normalize=False, return_length=False):
    vec_name = [0] * max_length
    num_shingles = len(name)-1
    for i in range(min(max_length, num_shingles)):
        try: 
            shingle = name[i:i+2]
            vec_name[i] = pairs.index(shingle)
        except: 
            print(name, name[i:i+2]) 
    vec_name = np.array(vec_name)
    if normalize: 
        vec_name = vec_name / 729.
    if return_length:
        return vec_name, num_shingles
    else: 
        return vec_name
         
def disembed_shingles(vec_name, pairs=pairs):
    name = [int(round(i)) for i in list(vec_name) if i != 0.] 
    decoded = ''
    for i in range(len(name)):
        try:
            decoded = decoded + pairs[name[i]][0]
        except:
            decoded = decoded + ' '
    decoded = decoded + pairs[name[i]-1][1]
    return decoded

def embed_letters(name, max_length, normalize=False, return_length=False):
    letters = 'abcdefghijklmnopqrstuvwxyz '
    vec_name = [0] * max_length
    num_letters = len(name)
    for i in range(min(max_length, len(name))):
        letter = name[i]
        vec_name[i] = letters.index(letter) + 1
    vec_name = np.array(vec_name)
    if normalize:
        vec_name = vec_name / 27.
    if return_length:
        return vec_name, num_letters
    else:
        return vec_name

In [623]:
K.clear_session() 
orig_dim = 20 
intermediate_dim = 128
latent_dim = 2
epochs = 50
batch_size = 16

namesA = np.vstack([embed_letters(name, max_length=orig_dim) for name in iowa_matches['lname1915']])
namesB = np.vstack([embed_letters(name, max_length=orig_dim) for name in iowa_matches['lname1915']])

namesA = namesA / np.max(namesA)
namesB = namesB / np.max(namesA)

x = Input(batch_shape=(batch_size, orig_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_sigma = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim))
    return z_mean + K.exp(z_log_sigma) * epsilon

z = Lambda(sampling)([z_mean, z_log_sigma])

decoder_h = Dense(intermediate_dim, activation='relu')
reconstruction = Dense(orig_dim, activation='sigmoid')

h_decoded = decoder_h(z)
x_decoded_mean = reconstruction(h_decoded)

# end-to-end autoencoder
vae = Model(x, x_decoded_mean)

# encoder, from inputs to latent space
encoder = Model(x, z_mean)

# generator, from latent space to reconstructed inputs
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

def vae_loss(x, x_decoded_mean):
    xent_loss = binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return xent_loss + kl_loss

vae.compile(optimizer='rmsprop', loss=vae_loss)
print(vae.summary())

ValueError: Input 0 is incompatible with layer dense_5: expected axis -1 of input shape to have value 64 but got shape (None, 128)

In [601]:
vae.fit(namesA, namesB, 
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=.2,
        callbacks=callbacks)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (16, 20)             0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (16, 64)             1344        input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (16, 2)              130         dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (16, 2)              130         dense_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (

KeyboardInterrupt: 